In [1]:
import folium

# Créer la carte
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Définir le HTML pour le marqueur avec un ID unique
marker_html = """
<div id="custom-marker" style="background-color: blue; width: 20px; height: 20px; border-radius: 50%; cursor: pointer;">
</div>
"""

# Ajouter le marqueur à la carte en utilisant un DivIcon avec le HTML défini
marker = folium.Marker([48.8566, 2.3522], icon=folium.DivIcon(html=marker_html)).add_to(
    m
)

# Injecter le script JavaScript pour changer la couleur du marqueur au clic
script = """
<script>
document.addEventListener("DOMContentLoaded", function() {
    var marker = document.getElementById("custom-marker");
    marker.addEventListener("click", function() {
        // Changer la couleur du marqueur
        if (marker.style.backgroundColor === "blue") {
            marker.style.backgroundColor = "red";
        } else {
            marker.style.backgroundColor = "blue";
        }
    });
});
</script>
"""

# Ajouter le script à la carte
m.get_root().html.add_child(folium.Element(script))

# Enregistrer la carte
m.save("map.html")

In [3]:
import pandas as pd
import streamlit as st
import folium
from streamlit_folium import st_folium

# Charger le DataFrame depuis un fichier Excel ou créer un nouveau DataFrame si le fichier n'existe pas
try:
    df = pd.read_excel("base_de_donnees.xlsx")
except FileNotFoundError:
    # Créer un dictionnaire avec des données fictives
    data = {
        "marker_id": [1, 2, 3, 4, 5],
        "bloqué": [1, 1, 0, 1, 0],
        "forcé": [0, 1, 1, 0, 0],
        "latitude": [48.8566, 48.8570, 48.8575, 48.8580, 48.8585],
        "longitude": [2.3522, 2.3525, 2.3530, 2.3535, 2.3540],
    }
    df = pd.DataFrame(data)


# Fonction pour mettre à jour les colonnes "bloqué" et "forcé" d'un nœud
def update_node(node_id):
    if node_id in df["marker_id"].values:
        # Mettre à jour les colonnes
        df.loc[df["marker_id"] == node_id, ["bloqué", "forcé"]] = 0, 0

        # Sauvegarder les modifications dans le fichier Excel
        df.to_excel("base_de_donnees.xlsx", index=False)
        return True
    return False


# Gérer la mise à jour à partir des requêtes GET
node_id_query = st.experimental_get_query_params().get("node_id", [None])[0]

if node_id_query is not None:
    # Convertir l'ID en entier pour la recherche
    node_id = int(node_id_query)
    success = update_node(node_id)
    if success:
        st.success("Les valeurs 'bloqué' et 'forcé' ont été mises à jour avec succès.")
    else:
        st.error("Échec de la mise à jour : nœud non trouvé.")

# Créer la carte
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajouter des marqueurs en fonction des nœuds de votre base de données
for index, row in df.iterrows():
    node_id = row["marker_id"]  # ID du nœud
    lat, lon = row["latitude"], row["longitude"]
    color = (
        "red" if row["bloqué"] else "blue"
    )  # Changer la couleur en fonction de l'état

    marker_html = f"""
    <div id="custom-marker-{node_id}" style="background-color: {color}; width: 20px; height: 20px; border-radius: 50%; cursor: pointer;">
    </div>
    """

    # Ajouter le marqueur
    marker = folium.Marker([lat, lon], icon=folium.DivIcon(html=marker_html)).add_to(m)

    # Injecter le script JavaScript pour gérer le clic sur le marqueur
    script = f"""
    <script>
    document.addEventListener("DOMContentLoaded", function() {{
        var marker = document.getElementById("custom-marker-{node_id}");
        marker.addEventListener("click", function() {{
            // Changer la couleur du marqueur
            var newColor = marker.style.backgroundColor === "blue" ? "red" : "blue";
            marker.style.backgroundColor = newColor;

            // Envoyer une requête à Streamlit pour mettre à jour la base de données
            const markerId = {node_id}; // Utilisation de l'ID du nœud ici
            fetch(`/?node_id=${markerId}`); // Correcte interpolation ici
        }});
    }});
    </script>
    """
    m.get_root().html.add_child(folium.Element(script))

# Afficher la carte dans Streamlit
st_folium(m)

2024-10-25 13:13:39.887 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-25 13:13:39.897 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-25 13:13:39.978 
  command:

    streamlit run /home/joe/Documents/GitHub/Ileo/env/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-25 13:13:39.980 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-25 13:13:39.982 Please replace `st.experimental_get_query_params` with `st.query_params`.

`st.experimental_get_query_params` will be removed after 2024-04-11.

Refer to our [docs page](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.query_params) for more information.


NameError: name 'markerId' is not defined